# Final Project: Analysis of Two Museum Datasets

Name(NetID): Jenna Kim(jkim682) | Yingying Han(yh17) | Xinyu Huang(xinyuh10) 



## Data

We used three different datasets:

1) Two meseum datasets: Metropolitan & Indianapolis  
2) World Bank dataset:  
3) Ethnicity dataset:


## Data Preprocessing

1) We integrated two meseum datasets  
2) We merged two World bank datasets   
3) We merged two ethnicity datasets

## Load Data

### Yingying & Xinyu, you can work on the following processed datasets
#### Note: I integrated three .py scripts we had created for collecting data. Please see "museum_data.py" file.

In [3]:
import pandas as pd

file1 = "museum.csv"
file2 = "world_bank.csv"
file3 = "ethnicity.csv"
    
museum_df = pd.read_csv(file1, index_col=False, low_memory=False)
population_df = pd.read_csv(file2, index_col=False, low_memory=False)
ethnicity_df = pd.read_csv(file3, index_col=False, low_memory=False)

In [12]:
museum_df.tail()
museum_df.shape   # (485632, 15)

# population_df.head()
population_df.shape   # (263, 5)

# ethnicity_df.head()
ethnicity_df.shape   # (109, 2)

(109, 2)

## Analysis 1 : Distribution of artifacts by geographical information

### Within USA (By States)

### Origin of Nations 

## Analysis 2: Distribution of artifacts by time